In [ ]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    GOOGLE_API_KEY = None
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )


# Define the file path for your ticket database
DB_PATH = "maintenance_log.csv"

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types
import pandas as pd
import json
from pydantic import BaseModel, Field
import random
from datetime import datetime
from google import genai

print("✅ ADK components imported successfully.")

In [ ]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [ ]:
 # --- 3. AGENT SYSTEM INSTRUCTION ---
AGENT_INSTRUCTION = """
ROLE: You are an AI-powered Maintenance Concierge Agent.
GOAL: Convert raw complaints into structured JSON maintenance tickets.

CATEGORIES:
- Plumbing, Electrical, AC / Cooling, Internet / WiFi, Furniture / Interior, Cleaning, Exterior / Outdoor, General Maintenance

URGENCY LEVELS:
- HIGH: Safety risk, fire, leak, total failure.
- MEDIUM: Faulty, inconvenient.
- LOW: Cosmetic, minor.

OUTPUT REQUIREMENTS:
- Output strictly valid JSON.
- Ticket ID format: TKT-xxxxx (random 5 digits).
- If input is unclear, set "error": "clarification_needed".

JSON SCHEMA:
{
 "ticket_id": "string",
 "issue_summary": "string",
 "category": "string",
 "priority": "High/Medium/Low",
 "recommended_action": "string",
 "technician_required": "Yes/No",
 "estimated_resolution_time": "string",
 "risk_level": "Critical/Warning/Safe",
 "notes_for_maintenance_team": "string"
}
"""

In [ ]:
# --- 4. CORE FUNCTIONS ---

def generate_ticket(complaint_text):
    """
    Sends the complaint to Gemini and returns a structured dictionary.
    """
    if not GOOGLE_API_KEY:
        print("API Key not found. Cannot generate ticket.")
        return {"error": "API Key Missing"}

    client = genai.Client(api_key=GOOGLE_API_KEY)
    
    try:
        response = client.models.generate_content(
            
        model="gemini-2.5-flash-lite",
         
            config=types.GenerateContentConfig(
                system_instruction=AGENT_INSTRUCTION,
                response_mime_type="application/json", # Enforce JSON
                temperature=0.1 # Low temp for deterministic structure
            ),
            contents=[complaint_text]
        )
        # Use json.loads() to convert the strict JSON string output into a Python dictionary
        return json.loads(response.text)
    except Exception as e:
        return {"error": "Processing Failed", "details": str(e)}

def save_to_database(ticket_data):
    """
    Appends the ticket to a CSV file using Pandas, creating the file if it doesn't exist.
    """
    if "ticket_id" not in ticket_data:
        return "Invalid ticket data, not saved."

    # Add timestamp for record keeping
    ticket_data['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create DataFrame from the new ticket
    df_new = pd.DataFrame([ticket_data])
    
    # Append to CSV using Pandas
    if os.path.exists(DB_PATH):
        # File exists: Append without headers
        df_new.to_csv(DB_PATH, mode='a', header=False, index=False)
    else:
        # File doesn't exist: Create with headers
        df_new.to_csv(DB_PATH, mode='w', header=True, index=False)
    
    return f"Ticket {ticket_data['ticket_id']} logged successfully."

**Execution and Verification**

This cell runs the primary logic, uses the example complaint, prints the structured output, and shows how to verify the saved data.

In [ ]:
# --- 5. EXECUTION LOOP ---

# Example user input simulating a request
user_input = "The air conditioner in the lobby is dripping water and making a loud grinding noise."

print(f"User Complaint: {user_input}\n")

# 1. Generate Ticket using the Gemini Agent
ticket_json = generate_ticket(user_input)

# 2. Display Result (The structured JSON output)
print("--- AGENT OUTPUT ---")
print(json.dumps(ticket_json, indent=2))

# 3. Save to DB
status = save_to_database(ticket_json)
print(f"\n--- SYSTEM LOG: {status} ---")

# --- 6. VERIFICATION (Optional) ---
# Display the current contents of the CSV file to confirm the save operation
print("\n--- DATABASE CHECK (CSV Contents) ---")
if os.path.exists(DB_PATH):
    df_db = pd.read_csv(DB_PATH)
    print(df_db.tail(5)) # Display the last 5 entries
else:
    print(f"Database file '{DB_PATH}' not found after saving attempt.")